Bootstrap Routes


In [ ]:
# | default_exp routes.bootstrap

In [ ]:
# | exporti
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

In [ ]:
# | export
async def get_bootstrap(
    auth: dmda.DomoFullAuth, ## only works with DomoFullAuth authentication, do not use TokenAuth
    debug_api: bool = False, 
    session: httpx.AsyncClient = None,
    return_raw: bool = False
) -> rgd.ResponseGetData:
    """get bootstrap data"""

    if auth.__class__.__name__ != 'DomoFullAuth':
        raise dmda.InvalidAuthTypeError(function_name='get_bootstrap',
                                        domo_instance=auth.domo_instance, 
                                        required_auth_type =  dmda.DomoFullAuth )

    # url = f"https://{auth.domo_instance}.domo.com/api/domoweb/bootstrap?v2Navigation=false"
    url = f"https://{auth.domo_instance}.domo.com/api/domoweb/bootstrap?v2Navigation=true"

    res = await gd.get_data(
        url=url, method="GET", auth=auth, debug_api=debug_api, session=session, is_follow_redirects = True
    )

    if res.response == '' and not return_raw:
        raise Exception('BSR_Features:  no features returned - is there a VPN?')

    return res


#### sample implementation of get_boostrap

In [ ]:
import os


auth = dmda.DomoTokenAuth(
    domo_instance='domo-community', 
    domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])

# auth = dmda.DomoFullAuth(
#     domo_instance='domo-community', domo_password = os.environ['DOJO_PASSWORD'], domo_username= os.environ['DOMO_USERNAME'])

try:
    await get_bootstrap(auth = auth, debug_api = False)
    
except dmda.InvalidAuthTypeError as e:
    print(e)

get_bootstrap: This API rquires DomoFullAuth at domo-community


In [ ]:
import os

full_auth = dmda.DomoFullAuth(
    domo_instance='domo-community', domo_password = os.environ['DOJO_PASSWORD'], domo_username= os.environ['DOMO_USERNAME'])

res = await get_bootstrap(auth = full_auth, debug_api = False)
res
# .response.keys()

ResponseGetData(status=200, response={'notifierConfig': {'apiKey': 'e640cbc9816caa81b971', 'channelName': 'prod1-mmmm-0012-0200', 'presenceChannelName': 'presence-prod1-mmmm-0012-0200', 'isEncrypted': False, 'host': 'api.pusherapp.com', 'socketHost': 'ws.pusherapp.com', 'socketPort': '80', 'socketSslPort': '443'}, 'currentUser': {'USER_ID': 612085674, 'USER_GROUP': 'mmmm-0012-0200', 'USER_FULLNAME': 'Oleksii Zakrevskyi', 'USER_NAME': 'zakrevskyi@gmail.com', 'DEFAULT_PAGE': '30507758', 'USER_ROLE': None, 'USER_ROLE_ID': 810756122, 'USER_LOCALE': 'en-US', 'USER_RIGHTS': None, 'SSO_USER': False, 'USER_PAGESETTING_KPISIZE': {}, 'UNIQUE_VISITOR_ID': '2FCDCC658791F1594EC51E001277CDC13081BE4A', 'DAYS_SINCE_USER_CREATED': 853}, 'data': {'landingPageId': '30507758', 'favorites': [], 'pages': [{'id': '-100000', 'title': 'Overview', 'locked': False, 'sharedView': False, 'pageVisible': True, 'virtualPage': True, 'isValidBadgePage': True, 'canAddCard': True, 'bibHiddenPage': False, 'hasAccess': Tru

In [ ]:
# | export
async def get_bootstrap_features(
    auth: dmda.DomoAuth, session: httpx.AsyncClient = None,
    debug_api: bool = False,
    return_raw: bool = False
) -> rgd.ResponseGetData:

    res = await get_bootstrap(auth=auth, session=session, debug_api=debug_api, return_raw=return_raw)

    if return_raw:
        return res

    if not res.is_success:
        return None

    res.response = res.response.get("data").get("features")
    return res


#### sample implementation of get_bootstrap_features

In [ ]:
import os
import pandas as pd

full_auth = dmda.DomoFullAuth(
    domo_instance='domo-community', domo_password=os.environ['DOJO_PASSWORD'], domo_username= os.environ['DOMO_USERNAME'])

res = await get_bootstrap_features(auth=full_auth, debug_api=False, return_raw = False)

pd.DataFrame(res.response[0:5])


,id,name,label,description,enabled,purchased,type,environment,owner
0,4,search,,,True,False,STANDARD,,
1,5,launcher,,,True,True,PREMIUM,,
2,9,profile-reminder,,,True,False,STANDARD,,
3,17,enableSwapDatasource,,,True,False,STANDARD,,
4,19,up,,,True,True,STANDARD,,


In [ ]:
# | export
async def get_bootstrap_pages(
    auth: dmda.DomoAuth, session: httpx.AsyncClient = None, debug_api: bool = False, return_raw: bool = False
) -> rgd.ResponseGetData:
    res = await get_bootstrap(auth=auth, session=session, debug_api=debug_api)

    if return_raw:
        return res
        
    if not res.is_success:
        return None

    res.response = res.response.get("data").get("pages")
    return res


#### sample get bootstrap pages

In [ ]:
import os
import pandas as pd

full_auth = dmda.DomoFullAuth(
    domo_instance='domo-community', domo_password=os.environ['DOJO_PASSWORD'], domo_username= os.environ['DOMO_USERNAME'])

res = await get_bootstrap_pages(auth=full_auth, debug_api=False, return_raw = False)

pd.DataFrame(res.response[0:5])


,id,title,locked,sharedView,pageVisible,virtualPage,isValidBadgePage,canAddCard,bibHiddenPage,hasAccess,hierarchy,separator,type,pageLocation,owner,owners,isOwner,childCount,children
0,-100000,Overview,False,False,True,True,True,True,False,True,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-100003,Favorites,False,False,True,True,NaN,False,False,True,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-100002,Shared,False,False,True,True,NaN,False,False,True,0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30507758,Welcome,True,True,True,NaN,NaN,True,False,True,0,NaN,page,0.0,587894148.0,"[{'id': 1814479647, 'type': 'GROUP', 'displayN...",True,0.0,NaN
4,1267639476,Community Solutions,True,True,True,NaN,NaN,True,False,True,0,NaN,page,1.0,587894148.0,"[{'id': 587894148, 'type': 'USER', 'displayNam...",False,3.0,"[{'id': '1759878295', 'title': 'Beast Modes', ..."


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
